# OpenAI OSS fine-tuning by Trelis
Advanced scripts available at [Trelis.com](https://Trelis.com/ADVANCED-fine-tuning)

*Based on the [OpenAI cookbook notebook](https://cookbook.openai.com/articles/gpt-oss/fine-tune-transfomers).*


Large reasoning models like **OpenAI o3** generate a *chain‑of‑thought* to improve the accuracy and quality of their responses.  
However, most of these models reason in English, even when a question is asked in another language.

In this notebook, we show how the open‑weight reasoning model **`openai/gpt-oss-20b`** can be fine‑tuned to reason effectively in multiple languages.  
We'll add a new **“reasoning language”** option to the model’s system prompt and apply supervised fine‑tuning with Hugging Face’s **TRL** library on a multilingual reasoning dataset.

**Outline**

1. **Setup** – install libraries  
2. **Prepare the dataset** – download & format  
3. **Prepare the model** – load, quantize & LoRA‑wrap  
4. **Fine‑tuning** – train with multilingual reasoning data  
5. **Inference** – generate reasoning responses in different languages  

When we're done you’ll have a multilingual reasoning model that can:  

* reason in **English, Spanish, French, Italian, or German**,  
* even mix languages – e.g. ask in Spanish, reason in German, answer in Spanish.

> **Example**

```
User:
    ¿Cuál es el capital de Australia?
Assistant reasoning:
    Okay, der Benutzer fragt nach der Hauptstadt Australiens. [...]
Assistant response:
    La capital de Australia es **Canberra**. [...]
```


## 1&nbsp;&nbsp;Setup

In [1]:
# Install PyTorch (CUDA 12.8 build)
!python -m pip install --upgrade pip
!pip install uv -qU

# !pip show torch

!uv pip install torch --index-url https://download.pytorch.org/whl/cu128 --system -q

# Install remaining dependencies
!uv pip install hf_transfer "trl>=0.20.0" "peft>=0.17.0" "transformers>=4.55.0" trackio --system -q

import os
os.environ["HF_TRANSFER"] = "1"

In [2]:
# # Log in to the Hugging Face Hub
# from huggingface_hub import notebook_login
# notebook_login()

## 2&nbsp;&nbsp;Prepare the dataset

In [3]:
from datasets import load_dataset

# Multilingual chain‑of‑thought dataset
dataset = load_dataset("HuggingFaceH4/Multilingual-Thinking", split="train")
dataset

Dataset({
    features: ['reasoning_language', 'developer', 'user', 'analysis', 'final', 'messages'],
    num_rows: 1000
})

In [4]:
# Look at the first training example
dataset[0]

{'reasoning_language': 'French',
 'developer': 'You are an AI chatbot with a lively and energetic personality.',
 'user': 'Can you show me the latest trends on Twitter right now?',
 'analysis': "D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.\n\nJe devrais préciser que les tendances Twitter évoluent rapidement et sont spécifiques à chaque région. Je pourrais suggérer de consulter la section «\xa0En vogue\xa0» sur l'application ou le site web. Aussi, l'utilisation de hashtags et le suivi d'utilisateurs pertinents pourraient être utiles. Il est important de souligner que les tendances varient selon la région et l'heure de la journée. Je devrais garder un ton amical et 

The **gpt‑oss** models use the *Harmony* response format to structure conversations:

| role       | purpose                                                         |
|------------|-----------------------------------------------------------------|
| developer  | custom system instructions                                      |
| user       | user input                                                      |
| assistant  | tool calls or responses                                         |
| analysis   | chain‑of‑thought                                                |
| final      | final answer for the end‑user                                   |

We convert these messages with `tokenizer.apply_chat_template()` so the model understands them.

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

messages = dataset[0]["messages"]
conversation = tokenizer.apply_chat_template(messages, tokenize=False)
print(conversation)

<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-06

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions

reasoning language: French

You are an AI chatbot with a lively and energetic personality.<|end|><|start|>user<|message|>Can you show me the latest trends on Twitter right now?<|end|><|start|>assistant<|channel|>analysis<|message|>D'accord, l'utilisateur demande les tendances Twitter les plus récentes. Tout d'abord, je dois vérifier si j'ai accès à des données en temps réel. Étant donné que je ne peux pas naviguer sur Internet ou accéder directement à l'API de Twitter, je ne peux pas fournir des tendances en direct. Cependant, je peux donner quelques conseils généraux sur la façon de les trouver.

Je devrais préciser que les tendances Twitter évoluent rapidement et sont spécifiques à chaque ré

## 3&nbsp;&nbsp;Prepare the model

In [6]:
import torch
from transformers import AutoModelForCausalLM, Mxfp4Config

quantization_config = Mxfp4Config(dequantize=True)
model_kwargs = dict(
    attn_implementation="eager",
    torch_dtype=torch.bfloat16, # float16 for colab [although will OOM on T4], bfloat16 for ampere, hopper or later
    quantization_config=quantization_config,
    use_cache=False,
    device_map="auto",
)

model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
!nvidia-smi

Wed Aug  6 08:32:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:D1:00.0 Off |                    0 |
| N/A   41C    P0            144W /  700W |   44331MiB /  81559MiB |      5%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [8]:
messages = [{"role": "user", "content": "¿Cuál es el capital de Australia?"}]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

output_ids = model.generate(input_ids, max_new_tokens=512)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


systemYou are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-06

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.user¿Cuál es el capital de Australia?assistantanalysisUser asks: "¿Cuál es el capital de Australia?" Spanish: "What is the capital of Australia?" The answer: Canberra. Provide succinct response.assistantfinalLa capital de Australia es Canberra.


### LoRA configuration

In [9]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules="all-linear",
    target_parameters=[
        "7.mlp.experts.gate_up_proj",
        "7.mlp.experts.down_proj",
        "15.mlp.experts.gate_up_proj",
        "15.mlp.experts.down_proj",
        "23.mlp.experts.gate_up_proj",
        "23.mlp.experts.down_proj",
    ],
)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(


trainable params: 15,040,512 || all params: 20,929,797,696 || trainable%: 0.0719


## 4&nbsp;&nbsp;Fine‑tuning

In [14]:
from trl import SFTConfig

training_args = SFTConfig(
    learning_rate=2e-4,
    gradient_checkpointing=True,
    # num_train_epochs=1,
    max_steps=4,
    logging_steps=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    max_length=2048,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine_with_min_lr",
    lr_scheduler_kwargs={"min_lr_rate": 0.1},
    output_dir="outputs/gpt-oss-20b-multilingual-reasoner",
    report_to="trackio",
    push_to_hub=True,
)

In [15]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
)

trainer.train()

Step,Training Loss
1,1.199400
2,1.146900
3,1.034300
4,1.128100


TrainOutput(global_step=4, training_loss=1.1271681785583496, metrics={'train_runtime': 67.9793, 'train_samples_per_second': 0.941, 'train_steps_per_second': 0.059, 'total_flos': 1.3156460233532928e+16, 'train_loss': 1.1271681785583496})

In [16]:
trainer.save_model(training_args.output_dir)
trainer.push_to_hub(dataset_name="HuggingFaceH4/Multilingual-Thinking")

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/RonanMcGovern/gpt-oss-20b-multilingual-reasoner/commit/b11db5557e81b984b91c81d0c7c4d476e211dc3e', commit_message='End of training', commit_description='', oid='b11db5557e81b984b91c81d0c7c4d476e211dc3e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RonanMcGovern/gpt-oss-20b-multilingual-reasoner', endpoint='https://huggingface.co', repo_type='model', repo_id='RonanMcGovern/gpt-oss-20b-multilingual-reasoner'), pr_revision=None, pr_num=None)

## 5&nbsp;&nbsp;Inference
**YOU MAY NEED TO RESTART THE KERNEL HERE TO CLEAR THE GPU**

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

model_kwargs = dict(attn_implementation="eager", torch_dtype="auto", use_cache=True, device_map="auto")
base_model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b", **model_kwargs).cuda()

peft_model_id = "outputs/gpt-oss-20b-multilingual-reasoner"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model = model.merge_and_unload()

REASONING_LANGUAGE = "German"
SYSTEM_PROMPT = f"reasoning language: {REASONING_LANGUAGE}"
USER_PROMPT = "¿Cuál es el capital de Australia?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

gen_kwargs = {"max_new_tokens": 512, "do_sample": True, "temperature": 0.6}
output_ids = model.generate(input_ids, **gen_kwargs)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

MXFP4 quantization requires triton >= 3.4.0 and triton_kernels installed, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:159: UserWarning: Unsupported layer type '<class 'transformers.models.gpt_oss.modeling_gpt_oss.GptOssExperts'>' encountered, proceed at your own risk.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


systemYou are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-06

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.developer# Instructions

reasoning language: Germanuser¿Cuál es el capital de Australia?assistantanalysisOk, der Benutzer hat gefragt: "¿Cuál es el capital de Australia?" Das bedeutet auf Spanisch: "Was ist die Hauptstadt von Australien?" Ich muss die Hauptstadt von Australien nennen. Ich erinnere mich, dass Canberra die Hauptstadt von Australien ist. Der Benutzer hat die Frage auf Spanisch gestellt, also sollte die Antwort auf Spanisch sein. Ich muss also "Canberra" auf Spanisch antworten. Der Benutzer hat nicht nach etwas anderem gefragt, also sollte die Antwort einfach sein. Ich muss sicherstellen, dass die Antwort korrekt ist und in Spanisch verfasst ist. Der Benutzer hat nach einer kurzen Antwort gefragt, also halte ich es kurz und bündig. 

Ich kann einfach 

In [5]:
# You need to train more for this to work in chinese.

REASONING_LANGUAGE = "French"
SYSTEM_PROMPT = f"reasoning language: {REASONING_LANGUAGE}"
USER_PROMPT = "C'est quoi la langue nationale du Canada?"

messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": USER_PROMPT},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
).to(model.device)

output_ids = model.generate(input_ids, **gen_kwargs)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

systemYou are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-06

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.developer# Instructions

reasoning language: FrenchuserC'est quoi la langue nationale du Canada?assistantanalysisOk, l'utilisateur demande quelle est la langue nationale du Canada. Je dois répondre en français puisque la requête est en français. Je dois confirmer que le Canada est officiellement bilingue, avec l'anglais et le français comme langues officielles. Puisque la question est "C'est quoi la langue nationale du Canada?", je pourrais répondre que le Canada est officiellement bilingue, donc les deux langues, l'anglais et le français, sont reconnues comme langues nationales. Je devrais mentionner que cela dépend de la région, car certaines provinces ont une langue officielle différente, comme le québécois au Québec. Je devrais également mentionner que le fran

## 6&nbsp;&nbsp;Conclusion

You fine‑tuned **`openai/gpt-oss-20b`** to reason in multiple languages using **TRL** + **LoRA** and the *Multilingual‑Thinking* dataset.  
Adapt these steps to your own data and build models that think in any language you need!

For more advanced scripts, check out [Youtube.com/@TrelisResearch].

---

*Notebook generated by Trelis.*